## Convolutional Nerual Network Flower Species Classifier

In [ ]:
# Imports here
import numpy as np
import torch
from torch import nn
from torch import optim
from torch.optim import lr_scheduler

import torchvision
from torchvision import datasets,models,transforms

import matplotlib.pyplot as plt
import os
import time
import copy

import json

%matplotlib inline
plt.ion()

### Data loading and model defining

Define hyper-parameters.

In [0]:
batch_size = 64
num_worker = 4
learn_rate = 0.002
N_epochs = 60

For the trainning images, apply transformations such as random scaling, cropping, and flipping. The helps to generalize the data and increase validation accuracy. To make use of pre-trained models from ImageNet, images has to be 224x224, with normalized means and standard deviations of [0.485, 0.456, 0.406] and [0.229, 0.224, 0.225] for [R,G,B],respectively.

In [0]:
data_dir = '/content/drive/My Drive/Colab Notebooks/flower_data'
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]), 
}
image_datasets = {
                x: datasets.ImageFolder(os.path.join(data_dir,x),data_transforms[x]) for x in ['train','valid']
}
dataloaders = {
                x: torch.utils.data.DataLoader(image_datasets[x], batch_size = batch_size, shuffle = True, num_workers = num_worker)
                for x in ['train','valid']
}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid']}

A pretrained Resnet152 is used to build the classifier model, the CNN part of the model is frozen, and the FC part of the model is redefined. The trainning process will only update the weights for the newly defined FC layers.

In [0]:
ClassifierModel = models.resnet152(pretrained = True)
for param in ClassifierModel.parameters():
    param.requires_grad = False
    
num_ftr = ClassifierModel.fc.in_features
ClassifierModel.fc = nn.Sequential(
                                    nn.Linear(num_ftr,1000),
                                    nn.ReLU(),
                                    nn.Dropout(0.2),
                                    nn.Linear(1000,102)
                                  )

criterion = nn.CrossEntropyLoss()

#optimizer = optim.Adam(ClassifierModel.classifier.parameters(),lr = learn_rate)
optimizer = optim.SGD(ClassifierModel.fc.parameters(),lr = learn_rate,momentum = 0.9)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=0.8*N_epochs, gamma=0.5)


In [0]:
#---------------------define device-----------------------#
device = torch.device("cuda" if torch.cuda.is_available() else "cpu" )
ClassifierModel.to(device)

### Training process

In [0]:
#------------------helper function : model train------------------#
def train_model(model,criterion, optimizer, scheduler, num_epochs):
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    batch = 0
    best_epoch = 0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-'*10)
        start_epoch = time.time()
        
        #each epoch has trainning and validation phase
        for phase in ['train','valid']:
            if phase == 'train':
                scheduler.step()
                model.train()
            else :
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0
            
            #iterate over data
            for inputs, labels in dataloaders[phase]:
                start_batch = time.time()
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                batch += 1
                #zero the parameter gradients
                optimizer.zero_grad()
                #forward prop.
                with torch.set_grad_enabled(phase=='train'):
                    outputs = model(inputs)
                    _,preds = torch.max(outputs,1)
                    loss = criterion(outputs,labels)
                    
                #backward prop.
                    if phase =='train':
                        loss.backward()
                        optimizer.step()
                        
                #statistics
                running_loss +=loss.item()*inputs.size(0)
                running_corrects +=torch.sum(preds == labels.data)
                #print('Batch {} : {}'.format(batch,time.time() - start_batch))
                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase,epoch_loss,epoch_acc))
                  #'Epoch_time:',time.time()-start_epoch)
                 
            
            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc :
                best_epoch = epoch
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
    print('Best val Acc: {:4f} at epoch {}'.format(best_acc,best_epoch+1))
            
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model
            

In [0]:
# ---------------------train the model to get a good classifier--------------------------#
ClassifierModel = train_model(ClassifierModel, criterion, optimizer, exp_lr_scheduler,
                       num_epochs=N_epochs)

### Inference Process

In [0]:
with open('/content/drive/My Drive/Colab Notebooks/cat_to_name.json', 'r') as f:
    class_names = json.load(f)
  
ClassifierModel.class_to_idx = image_datasets['train'].class_to_idx

In [0]:
#-------------------------save the model for resumming training or inference---------------------#
torch.save({
            'epoch': 60,
            'model_state_dict': ClassifierModel.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()
            
            }, '/content/drive/My Drive/Colab Notebooks/Classifier_Resnet152.pth.tar')

In [ ]:
#------------------------load the model for resumming training or inference-----------------------#
TestModel = models.resnet152(pretrained = True)
for param in TestModel.parameters():
    param.requires_grad = False
    
num_ftr = TestModel.fc.in_features
TestModel.fc = nn.Sequential(
                                    nn.Linear(num_ftr,1000),
                                    nn.ReLU(),
                                    nn.Dropout(0.2),
                                    nn.Linear(1000,102)
                                  )

TestOptimizer = optim.SGD(TestModel.fc.parameters(),lr = learn_rate,momentum = 0.9)

checkpoint = torch.load('/content/drive/My Drive/Colab Notebooks/Classifier_Resnet152.pth')
TestModel.load_state_dict(checkpoint['model_state_dict'])

TestModel.eval()


In [ ]:
from PIL import Image
test_data_dir = '/content/drive/My Drive/Colab Notebooks/flower_data/test'
pil_image = Image.open('/content/drive/My Drive/Colab Notebooks/flower_data/test/7/image_07216.jpg')
plt.imshow(pil_image)

In [ ]:
def process_image(pil_image):
    #resize the image with shorter side of 256 px
    width, height = pil_image.size
    if width <= height :
      pil_image.thumbnail((256*float(height/width),256*float(height/width)))
    else :
      pil_image.thumbnail((256*float(width/height),256*float(width/height)))
      
    # center crop the image with 224 px
    width, height = pil_image.size 
    left = np.ceil((width - 224)/2)
    top = np.ceil((height - 224)/2)
    right = np.ceil((width + 224)/2)
    bottom = np.ceil((height + 224)/2)
    pil_image = pil_image.crop((left, top, right, bottom))
    
    # To Tensor
    np_image = np.array(pil_image)/255
    
    # Normalize
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    np_image = (np_image - mean)/std
    
    # transpose the color channel
    np_image = np.transpose(np_image,(2,0,1))
    
    return np_image
    
  
np_image = process_image(pil_image)
#plt.imshow(test_pil)
print (np_image.shape)


In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax
  
imshow(np_image)

In [ ]:
#--------------------Class Prediction------------------------------
def predict(np_image, model, topk=5):
    torch_image = torch.from_numpy(np_image).float()
    torch_image = torch_image.unsqueeze(0)
    scores = model(torch_image)   
    m = nn.Softmax()
    probs = m(scores)
    Top_ps,Top_class = probs.topk(topk,dim=1)
    
    return Top_ps,Top_class
  
Tp_ps,Tp_cls = predict(np_image, TestModel)
print(Tp_ps)
print(Tp_cls)
